# Recipe Reviews — Predicting Star Ratings (1–5)

## Author: Jose Antonio Ayon Wu  
Master’s in Data Analytics, University of Niagara Falls (UNF)

## Project Overview & Executive Summary

**Goal.** Predict the exact star rating (1–5) for recipe reviews using `recipe_reviews.csv`, and explain which factors drive those predictions.

**Plan.** I’m keeping the workflow simple and reproducible: (1) load and clean the data, (2) quick EDA (distributions and correlations), (3) light feature engineering (e.g., log scales, text counts, categorical encoding), and (4) two models—Logistic Regression and Random Forest—with a small grid search. I evaluate with accuracy, precision/recall/F1, and AUC, plus confusion matrices and a short side-by-side comparison. Key figures are also exported as PNGs so the submission includes both the notebook and the visuals.

**What to expect.** Ratings are somewhat imbalanced (more 4–5s), so the middle classes will be trickier. I’ll call that out in the results and highlight which features matter most, along with a few practical takeaways and limitations.

## Block 1 – Data Loading & Initial Checks

**Goal of this section:**  
Load the dataset, check its structure, review missing values and duplicates, and confirm the **target column** (`stars`) for prediction.

**Notes:**  
- Several score-like columns exist (`likes_score`, `score_log`, `ranking_score`, etc.). These are features derived from user interactions, but **not the prediction target**.
- The true label is the **`stars` column**, representing recipe ratings from 0 to 5.
- This section only inspects the data—**no cleaning or transformation** is performed here. Data cleaning will be handled in the next block.

In [ ]:
# ============================================
# Part 4 — Block 1: Data Loading & Initial Checks
# ============================================

import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("data/recipe_reviews.csv")
print("[OK] Dataset loaded successfully")
print("Shape (rows, columns):", df.shape)

# Quick preview
print("\n[Head] First 5 rows:")
display(df.head())

# Column list and info
print("\n[Columns] List of columns:")
print(list(df.columns))

print("\n[Info] Dataset summary:")
df.info()

# Missing values
na_counts = df.isna().sum().sort_values(ascending=False)
print("\n[Missing values] Top 8 columns:")
display(na_counts.head(8))

# Bar plot of missing values (Top 6)
top_missing = na_counts.head(6)

if top_missing.sum() > 0:
    colors = ['salmon' if col == 'text' else 'lightblue' for col in top_missing.index]

    ax = top_missing.plot(kind="bar", color=colors)
    plt.title("Top 6 Columns by Missing Values")
    plt.ylabel("Number of NaN")
    plt.xlabel("Column")

    # Force integer ticks on Y axis
    import matplotlib.ticker as ticker
    ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig("outputs/missing_values_barplot.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
    plt.show()

# Target column = stars
rating_col = "stars"

print(f"\n[Target Column] Preview of '{rating_col}' distribution (raw values):")
display(df[rating_col].value_counts(dropna=False).sort_index())

ax = df[rating_col].value_counts().sort_index().plot(
    kind="bar", 
    color="skyblue", 
    edgecolor="black"
)
plt.title("Distribution of Star Ratings (Raw Data)", fontsize=14)
plt.xlabel("Star Rating", fontsize=12)
plt.ylabel("Number of Reviews", fontsize=12)
plt.savefig("outputs/stars_distribution_raw.png", dpi=300, bbox_inches="tight", pad_inches=0.1) 
plt.xticks(rotation=0)

# Add values on top of bars
for container in ax.containers:
    ax.bar_label(container, label_type="edge", fontsize=10)

plt.tight_layout()
plt.show()

# Duplicates
dup_count = df.duplicated().sum()
print(f"\n[Duplicates] Full-row duplicates detected: {dup_count}")

## Block 2 – Data Cleaning

**From the instructions (Part 4 – Comprehensive Case Study):**  
- *“Prepare the dataset for analysis by resolving common data quality issues.”*  
- *“Identify and handle missing values. In this dataset, the value '2' is sometimes used as a placeholder for missing data.”*  
- *“Remove or review duplicates and inconsistencies.”*  
- *“Ensure correct data types.”*  
- *“Normalize inconsistent categorical values.”*

**Steps in this section:**  
1. Handle missing values (including placeholders `'2'`).  
2. Drop rows with missing `text` (only 2 cases).  
3. Remove duplicates.  
4. Drop irrelevant identifier columns (`Unnamed: 0`, `comment_id`, `user_id`, `user_name`).  
5. Convert `created_at` into datetime format.  
6. Normalize categorical values (`region`, `device_type`).  
7. Verify distribution of target column (`stars`) after cleaning.

The goal is to leave the dataset **clean and ready for Exploratory Data Analysis (EDA)**.

In [ ]:
# ============================================
# Part 4 — Block 2: Data Cleaning
# ============================================

import numpy as np

# -----------------------------
# 1) Handle missing values
# -----------------------------
print("\n[Missing BEFORE cleaning]")
print(df.isna().sum().sort_values(ascending=False).head(8))

# replace placeholder "2" in text-like columns (object/string, excluding IDs)
obj_cols = df.select_dtypes(include="object").columns.tolist()
text_like_cols = [c for c in obj_cols if c not in ["comment_id", "user_id", "user_name"]]

for col in text_like_cols:
    df[col] = df[col].replace("2", np.nan).replace("", np.nan)

print("\n[Missing AFTER replacing placeholders]")
print(df.isna().sum().sort_values(ascending=False).head(8))

# drop rows where text is missing (only 2 rows)
df = df.dropna(subset=["text"])
print("\nDropped rows with missing 'text'. New shape:", df.shape)

# -----------------------------
# 2) Remove duplicates
# -----------------------------
dup_count = df.duplicated().sum()
print(f"\n[Duplicates BEFORE] Full-row duplicates: {dup_count}")
df = df.drop_duplicates()
print("[Duplicates AFTER] Shape:", df.shape)

# -----------------------------
# 3) Drop irrelevant identifier columns
# -----------------------------
drop_cols = ["Unnamed: 0", "comment_id", "user_id", "user_name"]
df = df.drop(columns=drop_cols, errors="ignore")
print("\nDropped identifier columns:", drop_cols)
print("Current columns:", list(df.columns))

# -----------------------------
# 4) Fix data types
# -----------------------------
# created_at as datetime
if "created_at" in df.columns:
    df["created_at"] = pd.to_datetime(df["created_at"], unit="s", errors="coerce")

# ensure stars is integer and valid (0–5)
df["stars"] = pd.to_numeric(df["stars"], errors="coerce")
df = df[df["stars"].isin([0,1,2,3,4,5])]
print("\n[Check] Unique values in stars:", sorted(df["stars"].unique()))

# -----------------------------
# 5) Normalize categorical values
# -----------------------------
for col in ["region", "device_type"]:
    if col in df.columns:
        df[col] = df[col].str.strip().str.lower()

print("\n[Preview] Unique values in region (normalized):")
if "region" in df.columns:
    print(df["region"].unique()[:10])  # show first 10 unique values

print("\n[Preview] Unique values in device_type (normalized):")
if "device_type" in df.columns:
    print(df["device_type"].unique())

# -----------------------------
# 6) Verify target distribution
# -----------------------------
print("\n[Stars distribution AFTER cleaning]")
print(df["stars"].value_counts().sort_index())

counts = df["stars"].value_counts().sort_index()

ax = counts.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Distribution of Star Ratings after Data Cleaning")
plt.xlabel("Star Rating")
plt.ylabel("Count")
plt.xticks(rotation=0)

# Add labels on top of bars
for i, v in enumerate(counts):
    plt.text(i, v + 100, str(v), ha="center", fontsize=9)

plt.tight_layout()
plt.savefig("outputs/stars_distribution_cleaned.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

## Block 3 – Exploratory Data Analysis (EDA)

**Notes**  
- In this block I perform Exploratory Data Analysis (EDA) to understand the structure of the dataset.  
- I include distributions of numeric and categorical variables, relation between features and the target (`stars`), and correlations.  
- The goal is not only to show plots but also to extract insights that will guide Feature Engineering and Model Building.  

In [ ]:
# =============================
# Block 3 – Exploratory Data Analysis (EDA)
# =============================
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

sns.set(style="whitegrid", palette="pastel")

# ------------------------------
# 0) Columns and helpers
# ------------------------------
num_cols = ["recipe_number", "recipe_code", "likes_score",
            "dislike_index", "response_level", "user_index"]
cat_cols = ["region", "device_type"]
target = "stars"

# ------------------------------
# 1) Distributions of numeric features (grouped) – Histogram + KDE
# ------------------------------
fig, axes = plt.subplots(2, 3, figsize=(18, 9))
fig.suptitle("Distributions of Numeric Features (Histogram + KDE)", fontsize=14)

BINS = 30  # consistent binning across charts

for ax, col in zip(axes.flatten(), num_cols):
    sns.histplot(
        data=df, x=col,
        bins=BINS, kde=True,
        color="blue", edgecolor="black",
        ax=ax
    )
    ax.set_title(f"Distribution of {col.replace('_',' ').title()}", fontsize=11)
    ax.set_xlabel(col.replace("_"," ").title())
    ax.set_ylabel("Count")
    ax.grid(axis="y", linestyle="--", alpha=0.6)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("outputs/numeric_distributions.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ------------------------------
# 2) Distributions of categorical features (grouped)
# ------------------------------
fig, axes = plt.subplots(1, len(cat_cols), figsize=(12, 4))
if len(cat_cols) == 1:
    axes = [axes]

for ax, col in zip(axes, cat_cols):
    order = df[col].value_counts().index
    sns.countplot(x=col, data=df, order=order,
                  color="skyblue", edgecolor="black", ax=ax)
    ax.set_title(f"Distribution of {col}", fontsize=11)
    ax.set_xlabel(col)
    ax.set_ylabel("Count")
    ax.grid(axis="y", linestyle="--", alpha=0.6)
    # Labels on each bar
    for container in ax.containers:
        ax.bar_label(container, fontsize=9)

plt.tight_layout()
plt.savefig("outputs/categorical_distributions.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ------------------------------
# 3) Numeric features vs target (boxplots with outliers)
# ------------------------------
pair_cols = ["recipe_number", "recipe_code", "likes_score", "dislike_index"]

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.ravel()

for i, col in enumerate(pair_cols):
    sns.boxplot(
        x=target, y=col, data=df,
        showfliers=True,   # keep outliers visible
        width=0.6, ax=axes[i]
    )
    axes[i].set_title(f"{col.replace('_',' ').title()} by Star Rating (with Outliers)", fontsize=11)
    axes[i].set_xlabel("Stars")
    axes[i].set_ylabel(col.replace("_"," ").title())
    axes[i].grid(axis="y", linestyle="--", alpha=0.6)
    axes[i].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.tight_layout()
plt.savefig("outputs/numeric_vs_stars_boxplots.png")
plt.show()

# ------------------------------
# 4) Target by categorical variables (grouped countplots)
# ------------------------------
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.countplot(x=target, hue="region", data=df, edgecolor="black", ax=axes[0])
axes[0].set_title("Star Ratings by Region", fontsize=11)
axes[0].set_xlabel("Stars")
axes[0].set_ylabel("Count")
axes[0].legend(title="region")
axes[0].grid(axis="y", linestyle="--", alpha=0.6)
axes[0].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

sns.countplot(x=target, hue="device_type", data=df, edgecolor="black", ax=axes[1])
axes[1].set_title("Star Ratings by Device Type", fontsize=11)
axes[1].set_xlabel("Stars")
axes[1].set_ylabel("Count")
axes[1].legend(title="device_type")
axes[1].grid(axis="y", linestyle="--", alpha=0.6)
axes[1].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.tight_layout()
plt.savefig("outputs/categorical_vs_stars_countplots.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ------------------------------
# 5) Correlation heatmap (numeric only)
# ------------------------------
plt.figure(figsize=(10, 7))
corr = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr, fmt='.2f', annot=False, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap of Numeric Features", fontsize=12)
plt.savefig("outputs/correlation_heatmap.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ------------------------------
# 6) Quick descriptive statistics
# ------------------------------
print("\n[Summary Statistics of numeric features]")
display(df.describe(include=[np.number]).T)


### Observations – Block 3 (EDA)

The dataset shows a very strong class imbalance, with most reviews concentrated in 5 stars.  
This imbalance is likely to dominate model predictions unless addressed in later steps.  

Continuous features such as `likes_score` and `dislike_index` display heavy right skewness, confirmed by KDE curves, suggesting that log transformations may help stabilize their scale.  
Boxplots with outliers highlight that extreme values appear mainly in likes- and dislikes-related variables.  

Categorical variables like `region` and `device_type` reveal broadly similar patterns, although imbalance persists within subgroups.  
Finally, the correlation heatmap suggests redundancy (e.g., `ranking_value` vs `ranking_score`), which motivates pruning during feature engineering.

## Block 4 – Feature Engineering

**From the instructions:**  
- *“Perform feature engineering to improve model performance. Create new variables, transformations, and encode categorical features.”*

**Rationale from EDA results:**  
- Numeric features are **highly right-skewed** (`likes_score`, `dislike_index`, etc.) → use `log1p`.  
- We detected **near-duplicate information** (r ≈ 0.99):  
  - (`likes_score` ↔ `likes`), (`dislike_index` ↔ `dislikes`), (`response_level` ↔ `responses`),  
    (`user_index` ↔ `user_score`), (`ranking_value` ↔ `ranking_score`), (`text_len` ↔ `word_count`).  
  - To **avoid multicollinearity** and keep the model simple, we will **keep one representative per par**.  
- Categorical features (`region`, `device_type`) look balanced → one-hot encoding is appropriate.  
- We keep the analysis **within course scope** (pandas / matplotlib / seaborn). No NLP avanzado.

**Steps**  
1) Text features (length and word count).  
2) Log-transform of skewed metrics.  
3) One-hot encoding for `region`, `device_type`.  
4) **Prune** highly correlated / redundant columns (keep one per pair).  
5) Build the **final modeling dataset** (`df_model`) and short sanity checks.

In [ ]:
# ==============================
# Block 4 – Feature Engineering
# ==============================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", palette="pastel")

# Start from the cleaned df (from Block 2)
data = df.copy()

# ---------------------------------
# 1) Simple text-derived features
# ---------------------------------
data["text_len"]    = data["text"].astype(str).apply(len)
data["word_count"]  = data["text"].astype(str).apply(lambda x: len(x.split()))
data["exclam_cnt"]  = data["text"].astype(str).str.count("!")  # simple signal, still in course scope

print("\n[Preview] New text features:")
display(data[["text", "text_len", "word_count", "exclam_cnt"]].head())

plt.figure(figsize=(8,5))
sns.histplot(data["word_count"], bins=30, kde=True)
plt.title("Distribution of word_count")
plt.tight_layout()
plt.savefig("outputs/word_count_distribution.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ---------------------------------
# 2) Log transform for skewed vars
# ---------------------------------
# We clip at 0 to avoid log of negatives; this is acceptable for these usage indices.
for col in ["likes_score", "dislike_index"]:
    data[col + "_log"] = np.log1p(data[col].clip(lower=0))

# quick before/after for likes_score
fig, ax = plt.subplots(1, 2, figsize=(12,5))
sns.histplot(data["likes_score"], bins=30, kde=True, ax=ax[0], color="red")
ax[0].set_title("Original likes_score")
sns.histplot(data["likes_score_log"], bins=30, kde=True, ax=ax[1], color="green")
ax[1].set_title("Log-transformed likes_score")
plt.tight_layout()
plt.savefig("outputs/likes_score_log_transformation.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ---------------------------------
# 3) One-hot encoding for categoricals
# ---------------------------------
encoded = pd.get_dummies(data, columns=["region", "device_type"], drop_first=True, dtype=int)
print("\n[Encoded columns added]:", [c for c in encoded.columns if c.startswith("region_") or c.startswith("device_type_")])

# ---------------------------------
# 4) Prune redundancy (keep one per highly correlated pair)
#    Decisions (guided by your correlation table):
#    - Keep the transformed versions + ratio-type signals.
#    - Drop raw counts / twin indices and one of the text length pair.
# ---------------------------------
drop_cols = [
    # keep likes_score_log + vote_ratio; drop raw twins
    "likes", "likes_score",
    # keep dislike_index_log; drop raw twin
    "dislikes", "dislike_index",
    # keep response_level; drop raw twin count
    "responses",
    # keep user_index; drop its twin
    "user_score",
    # keep ranking_value; drop twin
    "ranking_score",
    # text: keep word_count; drop text_len (r≈0.995)
    "text_len",
]

encoded = encoded.drop(columns=drop_cols, errors="ignore")

kept_example = ["likes_score_log", "dislike_index_log", "vote_ratio",
                "response_level", "user_index", "ranking_value", "word_count"]
print("\n[Kept representative signals]:", kept_example)

# (Optional sanity) check again extreme correlations (>0.95)
corr = encoded.corr(numeric_only=True)
high_again = [(a,b,float(corr.loc[a,b]))
              for a in corr.columns for b in corr.columns
              if a<b and abs(corr.loc[a,b])>0.95 and a!="stars" and b!="stars"]
print("\n[High correlations > 0.95 AFTER pruning]:")
print(high_again if high_again else "None detected")

# ---------------------------------
# 5) Final dataset for modeling
# ---------------------------------
# We will NOT model raw text now (no advanced NLP), only the simple text features created.
# So we keep 'text' for documentation, but exclude it from X later.
df_model = encoded.copy()

print("\n[Shape BEFORE FE]:", df.shape)
print("[Shape AFTER FE ]:", df_model.shape)

# quick head
display(df_model.head())

### Observations – Block 4 (Feature Engineering)

Feature engineering was guided directly by the EDA findings.  
Log transformation of `likes_score` and `dislike_index` reduced skewness, making these features less dominated by outliers.  

One-hot encoding was applied to `region` and `device_type`, ensuring categorical information could be captured by models.  
Highly correlated variables were pruned to avoid redundancy, reducing risk of multicollinearity.  

The resulting feature set is cleaner, more stable, and better suited for predictive modeling.

### Refinement of Feature Engineering  

Although the initial feature engineering step produced a dataset with 27 variables, further inspection revealed strong collinearity among several pairs (e.g., `likes_score` vs. `likes`, `dislike_index` vs. `dislikes`, `user_index` vs. `user_score`). Keeping both versions of such features could lead to redundancy and instability during model training.  

To address this, I refined the feature set by retaining only the representative signals (log-transformed versions, ratios, and normalized indexes) and removing duplicates. I also added `exclam_cnt` as a lightweight proxy for sentiment emphasis.  

As a result, the dataset was reduced to 20 features, eliminating correlations above 0.95 while preserving interpretability and predictive power. This refinement step ensures a cleaner and more robust dataset for the modeling stage.  

## Block 5 – Model Building

**From the instructions:**  
- *“Build appropriate models, document the process, and tune for performance.”*  

**Steps in this block:**  
1. Split the dataset into training and test sets (stratified by `stars`).  
2. Train a baseline **Logistic Regression** model.  
3. Train a **Random Forest** classifier as a more powerful model.  
4. Evaluate models with multiple metrics: accuracy, precision, recall, F1 (macro).  
5. Show confusion matrices to visualize errors.  
6. Discuss results and model selection.  

The goal is to compare a simple baseline with a more complex tree-based model, and assess which better captures the patterns in the data while handling the class imbalance.

In [ ]:
# =============================
# Block 5 – Model Building
# =============================

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score,   # <-- added for explicit Accuracy
)
import matplotlib.pyplot as plt

# ---------------------------------
# 1) Prepare X and y
# ---------------------------------
X = df_model.drop(columns=["stars", "text", "recipe_name", "created_at"])
y = df_model["stars"]

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("[Info] Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("[Distribution Train]", y_train.value_counts(normalize=True).round(3))
print("[Distribution Test ]", y_test.value_counts(normalize=True).round(3))

# ---------------------------------
# 2) Baseline Logistic Regression
# ---------------------------------
log_reg = LogisticRegression(max_iter=1000, class_weight="balanced", multi_class="multinomial")
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

print("\n[Logistic Regression Results]")
print(f"Accuracy (LogReg): {accuracy_score(y_test, y_pred_lr):.4f}")  # <-- explicit accuracy
print(classification_report(y_test, y_pred_lr, digits=3))

cm_lr = confusion_matrix(y_test, y_pred_lr)
ConfusionMatrixDisplay(cm_lr).plot(cmap="Blues")
plt.title("Confusion Matrix – Logistic Regression")
plt.tight_layout()  # <-- better spacing
plt.savefig("outputs/confusion_matrix_logistic_regression.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# ---------------------------------
# 3) Random Forest Classifier
# ---------------------------------
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\n[Random Forest Results]")
print(f"Accuracy (RF): {accuracy_score(y_test, y_pred_rf):.4f}")       # <-- explicit accuracy
print(classification_report(y_test, y_pred_rf, digits=3))

cm_rf = confusion_matrix(y_test, y_pred_rf)
ConfusionMatrixDisplay(cm_rf).plot(cmap="Greens")
plt.title("Confusion Matrix – Random Forest")
plt.tight_layout()  # <-- better spacing
plt.savefig("outputs/confusion_matrix_random_forest.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

#### Small hyperparameter search (kept short on purpose)

I ran a small grid over depth, estimators, and min_samples_split.  
**Best params (RF)**: `{'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 100}`.  
On the hold-out test set the tuned model reached **Accuracy = 0.759**. This is roughly on par with the baseline RF (0.763) and did not improve it—which can happen with small grids and limited signal. I kept the tuned version for the next section to compute feature importance.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import matplotlib.pyplot as plt

# --- Random Forest — small grid search (short on purpose) ---
rf_var = None
for name in ["best_rf", "rf_clf", "rf", "clf"]:
    if name in globals():
        rf_var = globals()[name]
        break

if rf_var is None or "X_train" not in globals() or "y_train" not in globals():
    print("[note] RF grid search skipped (need rf_clf/best_rf and X_train/y_train).")
else:
    # handle Pipeline vs bare estimator
    is_pipeline = hasattr(rf_var, "steps")
    p_n = "rf__n_estimators" if is_pipeline else "n_estimators"
    p_d = "rf__max_depth"     if is_pipeline else "max_depth"
    p_s = "rf__min_samples_split" if is_pipeline else "min_samples_split"

    rf_param_grid = {
        p_n: [100, 200],
        p_d: [None, 10, 20],
        p_s: [2, 5],
    }

    rf_gs = GridSearchCV(
        rf_var, rf_param_grid, cv=3, scoring="f1_weighted", n_jobs=-1, verbose=0
    )
    rf_gs.fit(X_train, y_train)
    best_rf = rf_gs.best_estimator_
    print("Best params (RF):", rf_gs.best_params_)

    # ---- Evaluate tuned model on hold-out test ----
    y_pred_rf_tuned = best_rf.predict(X_test)
    print(f"Accuracy (RF tuned): {accuracy_score(y_test, y_pred_rf_tuned):.4f}")
    print(classification_report(y_test, y_pred_rf_tuned, digits=3))

    # Confusion matrix (tuned)
    cm_rf = confusion_matrix(y_test, y_pred_rf_tuned)
    ConfusionMatrixDisplay(cm_rf).plot(cmap="Greens")
    plt.title("Confusion Matrix – Random Forest (tuned)")
    plt.tight_layout()
    plt.savefig("outputs/confusion_matrix_random_forest_tuned.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
    plt.show()

    # ROC curve (only if binary)
    if hasattr(best_rf, "predict_proba"):
        proba = best_rf.predict_proba(X_test)
        if proba.shape[1] == 2:
            fpr, tpr, _ = roc_curve(y_test, proba[:, 1])
            roc_auc = auc(fpr, tpr)
            plt.figure(figsize=(5.5, 4.5))
            plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
            plt.plot([0, 1], [0, 1], "--")
            plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC – Random Forest (tuned)")
            plt.legend(loc="lower right")
            plt.tight_layout()
            plt.savefig("outputs/roc_curve_random_forest_tuned.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
            plt.show()
        else:
            print("[note] Multiclass ROC not shown—sticking to CM + macro metrics for clarity.")

## Feature Importance (short and readable)
I computed MDI feature importance using the tuned Random Forest. This is a quick way to see what the model leaned on the most; it’s useful for intuition, but correlated features can split importance. I limited the plot to the Top-12 so it’s readable.

In [ ]:
# Feature importance
# If a tuned RF pipeline (best_rf) exists, use it. Otherwise, build a small one from df.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# 1) try to reuse an already-fitted pipeline (prefer my tuned RF)
pipe = None
source = None
if 'best_rf' in globals():
    pipe, source = best_rf, "best_rf"
elif 'rf_gs' in globals():
    try:
        pipe, source = rf_gs.best_estimator_, "rf_gs.best_estimator_"
    except Exception:
        pass
elif 'rf_clf' in globals():
    pipe, source = rf_clf, "rf_clf"

# 2) if nothing exists, build a small RF pipeline from df so this cell still works
if pipe is None:
    # make sure df exists (load from same folder if needed)
    if 'df' not in globals():
        df = pd.read_csv("data/recipe_reviews.csv")

    # infer the target column used in this course
    target_candidates = ["stars", "star_rating", "rating", "target"]
    target_col = next((c for c in target_candidates if c in df.columns), None)
    if target_col is None:
        raise RuntimeError("Couldn't infer the target column. Set target_col to your label (e.g., 'stars').")

    y = df[target_col]
    X = df.drop(columns=[target_col])

    # basic lists for preprocessing
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()

    preprocess = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
        ],
        remainder='drop'
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, stratify=y if getattr(y, "nunique", lambda: 0)() > 1 else None, random_state=42
    )

    # small RF so it runs fast but gives a reasonable importance picture
    pipe = Pipeline([
        ('prep', preprocess),
        ('rf', RandomForestClassifier(random_state=42, n_estimators=50, max_depth=10))
    ])
    pipe.fit(X_train, y_train)
    source = "ad-hoc RF (built here so the plot doesn't break)"

# 3) recover readable feature names from the ColumnTransformer (numeric + one-hot)
feature_names = []
try:
    ct = pipe.named_steps['prep']
    # try to find the numeric and categorical pieces regardless of the alias
    num_cols = None; cat_cols = None; ohe = None
    for name, tr, cols in ct.transformers_:
        # heuristic: StandardScaler in numeric branch; OneHotEncoder in categorical
        if hasattr(tr, "mean_") or name in ("num", "numeric"):
            num_cols = cols
        if "OneHotEncoder" in tr.__class__.__name__ or name in ("cat", "categorical"):
            cat_cols = cols; ohe = tr
    ohe_names = list(ohe.get_feature_names_out(cat_cols)) if ohe is not None else []
    feature_names = list(num_cols or []) + ohe_names
except Exception:
    feature_names = []

# 4) get importance values in this order: RF MDI -> |coef| -> permutation
importances = None
kind = ""
rf_step = pipe.named_steps.get('rf', None) if hasattr(pipe, "named_steps") else None
clf_step = pipe.named_steps.get('clf', None) if hasattr(pipe, "named_steps") else None

if rf_step is not None and hasattr(rf_step, "feature_importances_"):
    importances = rf_step.feature_importances_
    kind = "RandomForest MDI"
elif clf_step is not None and hasattr(clf_step, "coef_"):
    importances = np.abs(clf_step.coef_).ravel()
    kind = "|coef|"
else:
    # last resort: permutation (needs X_test / y_test)
    if 'X_test' in globals() and 'y_test' in globals():
        r = permutation_importance(pipe, X_test, y_test, n_repeats=5, random_state=42, n_jobs=-1)
        importances = r.importances_mean
        kind = "permutation"
    else:
        raise RuntimeError("No importances available (need RF/coef or X_test/y_test for permutation).")

# align names length or fallback
if not feature_names or len(feature_names) != len(importances):
    feature_names = [f"f{i}" for i in range(len(importances))]

# 5) plot top-12 and export
idx = np.argsort(importances)[::-1][:12]
plt.figure(figsize=(8, 4.5))
bars = plt.barh(range(len(idx)), importances[idx])
plt.yticks(range(len(idx)), np.array(feature_names)[idx])
plt.gca().invert_yaxis()
plt.title("Feature Importance — Top 12")
plt.xlabel(f"Importance ({kind}, higher = more influence)")
plt.ylabel("Feature (numeric + one-hot categories)")
try:
    plt.bar_label(bars, fmt="%.3f", padding=3)   # readable values
except Exception:
    pass
plt.tight_layout()
plt.savefig("outputs/feature_importance.png", dpi=300, bbox_inches="tight")
plt.show()


### Observations – Block 5 (Model Building)

Quick takeaways:
- **Logistic Regression** underperformed on this dataset (**Accuracy ≈ 0.318**), which fits the intuition: the decision boundaries are not linearly separable and we have many one-hot columns.
- **Random Forest (baseline)** did much better (**Accuracy ≈ 0.763**).
- The **tuned RF** via a short grid was **≈ 0.759**, very close but not higher than the baseline. With a larger search or different seeds we might squeeze a bit more, but I kept things intentionally short and reproducible.
- Confusion matrices show most mistakes in the middle classes (2–3 stars), which is common when the distribution skews toward 4–5.

### Observations – Block 6 (Balancing)

Two quick imbalance strategies:
- **Logistic Regression with class weights** came out at **Accuracy ≈ 0.318**, basically the same as the baseline LogReg.
- **Random Forest with SMOTE** reduced overall accuracy to **≈ 0.672**. That’s a typical trade-off: oversampling can help minority coverage/recall, but it often costs accuracy.

For ROC/AUC I show a one-vs-rest curve focused on **5-star** reviews to keep the plot readable. In a longer version I’d add macro OVR curves for all classes.

In [ ]:
# ============================================================
# Block 6 – Advanced Evaluation and Class Balancing
# ============================================================

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, accuracy_score
)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import numpy as np

print("[Block 6] Starting advanced evaluation with class balancing...")

# -------------------------------
# Logistic Regression (class weights)
# -------------------------------
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
print("[Class Weights]", class_weight_dict)

log_reg_bal = LogisticRegression(
    max_iter=1000, class_weight=class_weight_dict, solver="lbfgs", multi_class="multinomial"
)
log_reg_bal.fit(X_train, y_train)
y_pred_log_bal = log_reg_bal.predict(X_test)

print("\n[Logistic Regression (balanced) Results]")
print(f"Accuracy (LogReg balanced): {accuracy_score(y_test, y_pred_log_bal):.4f}")
print(classification_report(y_test, y_pred_log_bal, digits=3))

# -------------------------------
# Random Forest with SMOTE (train only)
# -------------------------------

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("[SMOTE] Resampled training set shape:", X_train_res.shape, y_train_res.shape)

print("[INFO] Training light RF for reproducibility (GitHub-safe)")
rf_bal = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=10)
rf_bal.fit(X_train_res, y_train_res)
y_pred_rf_bal = rf_bal.predict(X_test)

print("\n[Random Forest (SMOTE) Results]")
print(f"Accuracy (RF SMOTE): {accuracy_score(y_test, y_pred_rf_bal):.4f}")
print(classification_report(y_test, y_pred_rf_bal, digits=3))

# -------------------------------
# Confusion matrices (side by side)
# -------------------------------
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_log_bal, ax=axes[0], cmap="Blues")
axes[0].set_title("Confusion Matrix – Logistic Regression (Balanced)")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf_bal, ax=axes[1], cmap="Greens")
axes[1].set_title("Confusion Matrix – Random Forest (SMOTE)")
plt.tight_layout()
plt.savefig("outputs/confusion_matrices_balanced.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

# -------------------------------
# ROC curve — one-vs-rest for 5 stars
# -------------------------------
if hasattr(rf_bal, "predict_proba"):
    y_test_bin = (y_test == 5).astype(int)
    proba_rf = rf_bal.predict_proba(X_test)
    # find index of class '5' robustly
    cls_idx = list(rf_bal.classes_).index(5) if 5 in rf_bal.classes_ else None
    if cls_idx is not None:
        y_score_rf = proba_rf[:, cls_idx]
        fpr, tpr, _ = roc_curve(y_test_bin, y_score_rf)
        roc_auc = roc_auc_score(y_test_bin, y_score_rf)
        plt.figure(figsize=(7, 5))
        plt.plot(fpr, tpr, label=f"RF (SMOTE) AUC = {roc_auc:.2f}")
        plt.plot([0, 1], [0, 1], linestyle="--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC — Predicting 5 Stars vs Rest")
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig("roc_curve_random_forest_5stars.png", dpi=300, bbox_inches="tight", pad_inches=0.1)
        plt.show()
    else:
        print("[note] Class '5' not found in rf_bal.classes_; skipping the 5-star ROC.")
else:
    print("[note] rf_bal has no predict_proba; skipping ROC.")

## Executive Summary (Final Report & Business Conclusions)

**What worked best.** The **Random Forest (baseline)** was the top performer on the hold-out set (**Accuracy ≈ 0.763**). A short grid search produced a tuned RF (**≈ 0.759**), which was roughly on par but did not surpass the baseline. Logistic Regression lagged (**≈ 0.318**), which matches the idea that non-linear patterns dominate here.

**Why this matters.** Even with modest tuning, tree-based models handle sparse one-hot features and non-linearities better in this dataset. The feature-importance view is helpful to communicate what the model pays attention to, though I’d validate the top signals with permutation importance if I had more time.

**Imbalance.** Ratings skew toward higher values, so the middle classes (2–3 stars) are harder to predict consistently. Class weighting didn’t change LogReg much; SMOTE helped rebalance the training set but reduced overall accuracy, which is a common trade-off.

**Next steps.** If this were a production project I’d (a) expand the hyperparameter search (depth, leaves, min_samples), (b) test calibrated probabilities and macro-OVR curves for all classes, and (c) iterate on feature engineering (e.g., richer text signals, interactions) to target the hard middle classes.